In [29]:
import pandas as pd
import numpy as np
from functools import reduce

In [4]:
metadata = pd.read_csv("~/Desktop/BNFO285_Projects/Data/processed/processed_metadata.txt", sep="\t", header=0)
patient_list = pd.read_csv("~/Desktop/BNFO285_Projects/project_5/Amy/patients.txt", sep="\t", header = None)
valid_pateints = patient_list[0]
metadata = metadata[metadata["patient_id"].isin(valid_pateints)]
metadata

,patient_id,age,OS,OS_time,DSS,DSS_time,DFI,DFI_time,gender,tumor_site,tumor_class,tumor_stage,hpv_status
0,TCGA-4P-AA8J,66.0,0.0,102.0,0.0,102.0,NaN,NaN,MALE,tongue,primary,4.0,negative
1,TCGA-BA-4074,69.0,1.0,462.0,1.0,462.0,NaN,NaN,MALE,tongue,primary,4.0,NaN
2,TCGA-BA-4075,49.0,1.0,283.0,1.0,283.0,NaN,NaN,MALE,tongue,primary,3.0,NaN
3,TCGA-BA-4076,39.0,1.0,415.0,1.0,415.0,NaN,NaN,MALE,Larynx,primary,NaN,NaN
4,TCGA-BA-4077,45.0,1.0,1134.0,1.0,1134.0,NaN,NaN,FEMALE,tongue,primary,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,TCGA-UF-A7JS,59.0,1.0,680.0,1.0,680.0,NaN,NaN,MALE,tongue,primary,4.0,NaN
495,TCGA-UF-A7JT,72.0,1.0,993.0,1.0,993.0,NaN,NaN,FEMALE,mouth,metastasis,4.0,NaN
497,TCGA-UP-A6WW,58.0,0.0,518.0,0.0,518.0,NaN,NaN,MALE,tongue,primary,NaN,NaN
498,TCGA-WA-A7GZ,58.0,1.0,625.0,0.0,625.0,0.0,625.0,MALE,mouth,primary,NaN,NaN


In [45]:
threshold = 3 * 365  # 1825 days

# Conditions:
# 1) OS_time ≥ 1825 → patient “survived” at least 3 years (regardless of whether they later died)
# 2) OS_time < 1825 AND OS == 1 → patient died before 3 years → “did not survive” 3 years
# 3) OS_time < 1825 AND OS == 0 → censored before 3 years → status unknown (we’ll set to NaN)

conditions = [
    (metadata["OS_time"] >= threshold),
    (metadata["OS_time"] < threshold) & (metadata["OS"] == 1)
]
choices = [1, 0]

metadata["three_year_status"] = np.select(conditions, choices, default=np.nan)
three_year_metadata = metadata[["patient_id", "three_year_status"]].drop_duplicates()
three_year_metadata.dropna(subset=["three_year_status"], inplace=True)
three_year_metadata

,patient_id,three_year_status
1,TCGA-BA-4074,0.0
2,TCGA-BA-4075,0.0
3,TCGA-BA-4076,0.0
4,TCGA-BA-4077,1.0
5,TCGA-BA-4078,0.0
...,...,...
492,TCGA-UF-A7JK,0.0
493,TCGA-UF-A7JO,0.0
494,TCGA-UF-A7JS,0.0
495,TCGA-UF-A7JT,0.0


In [46]:
supp_metadata = pd.read_csv("/Users/clairez/Desktop/BNFO285_Projects/Data/processed/sample_metadata_enriched.tsv", sep="\t", header=0)
supp_metadata = supp_metadata[supp_metadata["patient_id"].isin(valid_pateints)]
supp_metadata = supp_metadata[["patient_id", "tobacco_smoking_history_indicator", "alcohol_history_documented"]]
supp_metadata["alcohol_history_documented"] = supp_metadata["alcohol_history_documented"].replace({"NO": 0, "YES": 1})
supp_metadata["tobacco_smoking_history_indicator"] = supp_metadata["tobacco_smoking_history_indicator"].replace({"Lifelong Non-smoker": 0, "Current smoker": 1,
                                                                                                                 "Current reformed smoker for < or = 15 years": 1,
                                                                                                                 "Current reformed smoker for > 15 years": 1,})
supp_metadata = supp_metadata.drop_duplicates()
supp_metadata

/var/folders/_k/b8vsv8097ss3sxnsdwkx1y_80000gn/T/ipykernel_33409/1025332672.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  supp_metadata["alcohol_history_documented"] = supp_metadata["alcohol_history_documented"].replace({"NO": 0, "YES": 1})
/var/folders/_k/b8vsv8097ss3sxnsdwkx1y_80000gn/T/ipykernel_33409/1025332672.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  supp_metadata["tobacco_smoking_history_indicator"] = supp_metadata["tobacco_smoking_history_indicator"].replace({"Lifelong Non-smoker": 0, "Current smoker": 1,


,patient_id,tobacco_smoking_history_indicator,alcohol_history_documented
0,TCGA-4P-AA8J,NaN,NaN
1,TCGA-BA-4074,1.0,1.0
2,TCGA-BA-4075,1.0,1.0
3,TCGA-BA-4076,1.0,1.0
4,TCGA-BA-4077,1.0,1.0
...,...,...,...
538,TCGA-UF-A7JS,NaN,NaN
539,TCGA-UF-A7JT,NaN,NaN
541,TCGA-UP-A6WW,NaN,NaN
542,TCGA-WA-A7GZ,NaN,NaN


In [47]:
cnv_features = pd.read_csv("/Users/clairez/Desktop/BNFO285_Projects/project_5/Wenshu/CNV_features.tsv", sep="\t", header=0)
cnv_features

,patient_id,WDR5
0,TCGA-4P-AA8J,0
1,TCGA-BA-4074,0
2,TCGA-BA-4075,0
3,TCGA-BA-4076,0
4,TCGA-BA-4077,1
...,...,...
343,TCGA-UF-A7JS,1
344,TCGA-UF-A7JT,1
345,TCGA-UP-A6WW,1
346,TCGA-WA-A7GZ,0


In [48]:
mutation_features = pd.read_csv("/Users/clairez/Desktop/BNFO285_Projects/project_5/Dan/train_mutation_embedding.csv", sep=",", header=0)
bool_cols = mutation_features.select_dtypes(include='bool').columns
mutation_features[bool_cols] = mutation_features[bool_cols].astype(int)
mutation_features

,patient_id,RELN,LRP1B,NOTCH1,PTPRD,PIK3CA,TP53,FBXW7,CDKN2A,NFE2L2,FAT1
0,TCGA-4P-AA8J,0,1,0,0,0,0,0,0,0,0
1,TCGA-BA-4074,0,0,0,0,0,0,0,0,0,0
2,TCGA-BA-4075,0,0,1,0,0,0,0,0,0,0
3,TCGA-BA-4076,0,0,0,0,0,1,0,0,0,0
4,TCGA-BA-4077,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
351,TCGA-UF-A7JS,0,0,0,0,0,1,0,0,0,0
352,TCGA-UF-A7JT,0,0,0,0,0,0,0,0,0,0
353,TCGA-UP-A6WW,0,0,0,0,0,0,0,0,0,0
354,TCGA-WA-A7GZ,0,0,0,0,0,1,0,0,0,0


In [49]:
rna_features = pd.read_csv("/Users/clairez/Desktop/BNFO285_Projects/project_5/Harrison/PCA_subset.tsv", sep="\t", header=0)
rna_features

,patient_id,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40
0,TCGA-DQ-5625,57.402773,12.534579,-12.364469,-8.583862,6.249989,-10.650234,-9.773156,-8.710869,-11.876764,...,-1.869929,-4.619362,0.366654,1.474022,-2.103360,-1.509451,-0.894887,-1.723013,-0.412455,-2.494964
1,TCGA-CV-6956,27.996105,17.050628,0.064888,-10.638154,-25.494637,8.254441,-15.769213,13.685947,-5.884615,...,-1.882987,-4.782358,3.190577,2.616468,-6.325822,1.668413,-0.043877,-0.914949,2.511687,-5.679988
2,TCGA-CV-7250,-25.969790,8.837039,17.580533,-10.430295,-7.345048,-8.779846,-10.612032,6.945576,3.487698,...,-1.937019,-1.090919,3.498057,0.848722,-2.433595,4.253617,1.268875,-1.046241,3.865996,1.649350
3,TCGA-BA-6872,-21.425448,3.103690,-20.780957,17.195466,8.532032,21.312102,2.857813,-4.308064,-9.154299,...,3.656153,0.682178,3.621453,0.854258,-8.233721,-1.661506,-1.586596,-1.711933,-1.214834,-0.646028
4,TCGA-CV-A6JZ,-12.034730,-17.045342,-27.368681,2.877659,20.207160,-9.356129,10.246727,-4.096803,-13.254286,...,-1.177277,2.439232,-5.160661,2.272224,-4.819804,8.208353,-1.009656,-3.011053,2.264090,-3.835137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,TCGA-CV-7102,48.428656,-27.252157,-8.830460,9.171790,-3.864963,8.114416,-12.906079,-6.174674,-0.850140,...,9.693010,-0.935301,-0.552000,3.035344,3.022518,4.100101,-3.567778,9.245594,-0.528889,1.256739
352,TCGA-DQ-7592,30.413308,9.164937,-6.721484,5.523265,-6.536309,-4.481993,-7.702960,3.681352,-10.383822,...,-0.291755,2.227043,-7.003011,-4.009349,1.572050,3.466296,5.586276,-2.941945,-5.923357,-3.106166
353,TCGA-CV-A6JD,-9.670388,-35.057530,-39.867281,-3.805781,18.403374,-0.765926,-2.877192,-6.982164,7.152203,...,-7.623640,6.840023,-2.544782,-2.677370,8.174533,-3.928624,-3.928740,-4.587826,-6.038613,3.181007
354,TCGA-CQ-A4CI,-25.770909,-41.223657,2.764470,0.990855,-37.563371,-0.838564,9.464249,-8.439206,3.855538,...,-3.762358,-5.564598,7.071308,-6.471262,-1.893580,-5.496762,3.831099,4.824374,-8.111654,-2.579510


In [52]:
dfs = [three_year_metadata, supp_metadata, cnv_features, mutation_features, rna_features]
merged_feature_table = reduce(
    lambda left, right: pd.merge(left, right, on='patient_id', how='inner'),
    dfs
)
merged_feature_table["three_year_status"].sum()

np.float64(87.0)

In [51]:
merged_feature_table.to_csv("/Users/clairez/Desktop/BNFO285_Projects/project_5/Wenshu/merged_feature_table.csv", index=False, header=True)